In [ ]:
# 영문 분석 -> 워드 클라우드로 그리기(시각화)

## 샘플 데이터 영문 학술 문서의 제목만 추출, 그 단어의 빈도 분석 시각화 
## 데이터 수집 : Big data  키워드로 검색 후 , 해당 학술 연구 정보 서비스에서 수집 해보기
## 조합, pandas.concat(), 정제 re 정규식, 기본적인 유효성 체크
## 변환 : word_tokenize(), lower()
## matplotlib.pyplot 이용하기 
## 단어 빈도 구해주는 Counter() 이용

## 비정형 빅데이터 분석을 말하고 -> 자연어 처리 (nature language processing)
## 자연어 처리 예) 음성, 텍스트 정보 추출

# 관련 단어 용어 정리
## 텍스트 분석: 자연어 처리와 데이터 마이닝 결합하여 발전되었고
## 비정형 텍스트 데이터에서 정보를 추출하는 분석 방법
## 분석 방법 1) 텍스트 분류 2) 텍스트 군집화 3) 감성 분석

## 전처리 : 분석 작업의 정확도를 높이기 위해서 사용할 데이터를 정리하고 변환하는 작업 
### 수행하는 작업
### 정제 (cleaning) : 불필요한 기호, 문자 필터하는 작업, 정규식을 이용해서 작업을 함 
### 정규화 ( normallization) : 형대가 다른 단어를 특정의 형태로 변환 작업, 대문자, 소문자 통합하는 작업, 의미가 비슷한 단어끼리 통합작업
### 토큰화 ( tokennization) : 토큰으로 정하는 기본 단위로 분리 작업. 문장 기준 , 단어 기준이 될 수도 있다.
### 불용어제거 ( stopword ) : 의미 있는 단어를 추출하기 위해서, 조사, 관사, 접미사, 접두사 등 제거하는 작업
### 어간 추출 ( semming) : 단수, 복수, 진행형(시제), 분리하는 작업
### 표제어 추출 (lemmatization) : 단어의 기본형 형태로 일반화 하는 작업
### 예) gone -> go , am -> be , going -> go 

### 워드클라우드 : 텍스트 분석에서 빈도를 시각화 할 때 많이 사용됨.

In [6]:
# 데이터 수집
# 한국교육학술정보원(KERIS)의 RISS 사이트
# https://www.riss.kr/index.do
# Big data 검색해보기.
# 한 페이지당 100개씩 내보내기 엑셀 파일 간략 정보, 반복 10번
# 1000개의 데이터에서 제목만 추출 및 분류 작업하기

In [5]:
# !pip3 install wordcloud
!pip3 install nltk



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


NameError: name 'nltk' is not defined

In [7]:
import nltk
nltk.download()

2023-12-19 12:29:49.790 Python[34552:4393428] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [14]:
# 관련 패키지들 임포트
import pandas as pd 

# 경로 이름 지정해서 파일 처리할 때 사용하는 도구
import glob 
# 정규 표현식에 사용하는 도구 
import re 
# 2차원 리스트를 -> 1차원 리스트로 차원 축소시 사용하는 도구 
from functools import reduce
# 자연어 처리 패키지 중에서, 단어 토큰화 작업.
from nltk.tokenize import word_tokenize
# 불용어 처리 작업. 
from nltk.corpus import stopwords 
# 표제어 추출 
from nltk.stem import WordNetLemmatizer 
# 단어의 빈도를 추출하는 도구. 
from collections import Counter 
import matplotlib.pyplot as plt
# 단어의 빈도수를 시각화하는 도구, 빈도가 높을수록 글자 크기가 커짐. 
from wordcloud import STOPWORDS, WordCloud 

# 데이터 조합(병합)하기
# 현재 폴더 하위에 , 받았던 엑셀 파일명 10개 선택하기
all_files = glob.glob("./myCabinetExcelData*.xls")
all_files 

# 엑셀 파일 읽어서  -> 데이터 브레임 (표형태) 변환 -> 특정 리스트에 담아두기
# 임시로 저장할 리스트 변수
all_files_data = []

# all_files 에 담겨진 엑셀 파일의 위치가 드렁있고, 
# 해당 위치의 엑셀 파일ㅇ르 읽어서, 데이터 프레임 표 형태로 변환하기
# 임시 리스트에 담기 
for file in all_files:
  # 해당 엑셀 파일의 위치의 물리 파일 읽기
  data_frame = pd.read_excel(file)
  # 임시 리스트에 담기
  all_files_data.append(data_frame)

# 샘플 확인 해보기, 첫번째 요소 확인 해보기
# all_files_data = [엑셀1,엑셀2,엑셀3,...]
all_files_data[0]

# 오류 발생 
# pip3 install xlrd

# 엑셀 파일 10개를 병합해서 출력해보기 
# axis=0 , 세로 방향으로 , 밑으로 데이터를 붙이는 작업
all_files_data_concat = pd.concat(all_files_data, axis=0, ignore_index =True)
all_files_data_concat.shape

# 병합된 파일을 csv파일로 변환하기 
all_files_data_concat.to_csv("/Users/user/BigDataTest/My_Python/2-자연어처리-영문-한글-통계/riss_Bigdata.csv", encoding = "utf8", index = False)